In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
org_data = pd.read_csv("rule_based_data.csv",index_col=False)
if "Unnamed: 0" in org_data.columns:
    org_data = org_data.drop(columns=["Unnamed: 0"])

In [4]:
org_data

,CROPS,TYPE_OF_CROP,SOIL,SOIL_PH,CROP_DURATION,TEMP,WATER_SOURCE,WATER_REQUIRED,RELATIVE_HUMIDITY
0,rice,cereals,Alluvia or loamy and clayey soil,6.4,150,39.91,"irrigated,rainfall",2242,79.09
1,rice,cereals,Alluvia or loamy and clayey soil,5.0,150,31.95,"irrigated,rainfall",906,79.17
2,rice,cereals,Alluvia or loamy and clayey soil,7.5,150,26.04,"irrigated,rainfall",2200,69.15
3,rice,cereals,Alluvia or loamy and clayey soil,6.4,150,20.87,"irrigated,rainfall",2390,75.78
4,rice,cereals,Alluvia or loamy and clayey soil,5.4,150,22.85,"irrigated,rainfall",1644,77.28
...,...,...,...,...,...,...,...,...,...
565,small onion,bulbvegetables,Sandy loam,6.3,77,17.68,"irrigated,rainfall",694,72.43
566,small onion,bulbvegetables,Sandy loam,6.5,87,19.25,"irrigated,rainfall",716,67.93
567,small onion,bulbvegetables,Sandy loam,6.2,72,23.80,"irrigated,rainfall",699,69.90
568,small onion,bulbvegetables,Sandy loam,6.2,84,23.39,"irrigated,rainfall",660,71.31


In [9]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

synthetic_df=pd.DataFrame()
for i in range(57):
    print(i)
    if i==0:
        crop_data = org_data[(i*10):(i+1)*10]
    else:
        crop_data = org_data[(i*10)+1:(i+1)*10]
    # Extract numerical columns
    numerical_columns = ["SOIL_PH", "CROP_DURATION", "TEMP", "WATER_REQUIRED", "RELATIVE_HUMIDITY"]
    categorical_columns = ["CROPS", "WATER_SOURCE", "SOIL","TYPE_OF_CROP"]

    # Create a MinMaxScaler
    scaler = MinMaxScaler()
    # Normalize only numerical columns
    crop_data[numerical_columns] = scaler.fit_transform(crop_data[numerical_columns])

    # Perform one-hot encoding for categorical columns
    encoder = OneHotEncoder(sparse=False, drop='first')
    categorical_encoded = encoder.fit_transform(crop_data[categorical_columns])
    categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names(categorical_columns))

    # Concatenate the encoded categorical columns with the normalized numerical columns
    normalized_data = pd.concat([crop_data[numerical_columns], categorical_encoded_df], axis=1)
    # GAN parameters
    latent_dim = 100
    num_samples = len(normalized_data)
    epochs = 10
    batch_size = 64
    # Generator model
    generator = keras.Sequential([
        layers.Input(shape=(latent_dim,)),
        layers.Dense(256, activation="relu"),
        layers.Dense(512, activation="relu"),
        layers.Dense(len(normalized_data.columns), activation="sigmoid")  # Output layer with same dimensions as input
    ])
    # Discriminator model
    discriminator = keras.Sequential([
        layers.Input(shape=(len(normalized_data.columns),)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(1, activation="sigmoid")  # Binary classification output
    ])
    # GAN model (combining generator and discriminator)
    discriminator.compile(loss="binary_crossentropy", optimizer="adam")
    discriminator.trainable = False
    gan_input = keras.Input(shape=(latent_dim,))
    gan_output = discriminator(generator(gan_input))
    gan = keras.Model(gan_input, gan_output)
    gan.compile(loss="binary_crossentropy", optimizer="adam")

    # Training loop
    for epoch in range(epochs):
        noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
        generated_data = generator.predict(noise)

        real_data_indices = np.random.choice(len(crop_data), batch_size)
        real_data = crop_data.drop(columns=["CROPS","WATER_SOURCE","SOIL","TYPE_OF_CROP"]).iloc[real_data_indices].values.astype(np.float32)

        real_data_label = np.ones((batch_size, 1), dtype=np.float32)
        fake_data_label = np.zeros((batch_size, 1), dtype=np.float32)

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_data, real_data_label)
        d_loss_fake = discriminator.train_on_batch(generated_data, fake_data_label)

        # Train generator (via GAN model)
        noise = np.random.normal(0, 1, size=(batch_size, latent_dim)).astype(np.float32)
        g_loss = gan.train_on_batch(noise, real_data_label)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}: D Loss Real: {d_loss_real:.4f}, D Loss Fake: {d_loss_fake:.4f}, G Loss: {g_loss:.4f}")
    
    # Generate synthetic crop data
    num_synthetic_samples = 100
    noise = np.random.normal(0, 1, size=(num_synthetic_samples, latent_dim))
    synthetic_data = generator.predict(noise)
    synthetic_data_denormalized = (synthetic_data * (scaler.data_max_ - scaler.data_min_)) + scaler.data_min_
    
    for col in normalized_data.columns:
        col_min = org_data[col].min()
        col_max = org_data[col].max()
        synthetic_data_denormalized[:, normalized_data.columns.get_loc(col)] = np.clip(
            synthetic_data_denormalized[:, normalized_data.columns.get_loc(col)], col_min, col_max
        )
    
    decimal_places = {"SOIL_PH": 1, "CROP_DURATION": 0, "TEMP": 2, "WATER_REQUIRED": 0, "RELATIVE_HUMIDITY": 2}
    synthetic_data_denormalized_rounded = synthetic_data_denormalized.copy()
    for column, places in decimal_places.items():
        synthetic_data_denormalized_rounded[:, normalized_data.columns.get_loc(column)] = np.round(synthetic_data_denormalized[:, normalized_data.columns.get_loc(column)], places)
    

    epoch_synthetic_df = pd.DataFrame(synthetic_data_denormalized_rounded, columns=normalized_data.columns)
    epoch_synthetic_df.insert(0, 'CROPS', org_data["CROPS"][(10*i)+1])
    epoch_synthetic_df.insert(1, 'TYPE_OF_CROP', org_data["TYPE_OF_CROP"][(10*i)+1])
    epoch_synthetic_df.insert(2, 'SOIL', org_data["SOIL"][(10*i)+1])
    epoch_synthetic_df.insert(5, 'WATERSOURCE', org_data["WATER_SOURCE"][(10*i)+1])
    synthetic_df = synthetic_df.append(epoch_synthetic_df, ignore_index=True)
    

0
2/2 [==============================] - 0s 4ms/step


c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 0: D Loss Real: 0.6874, D Loss Fake: 0.7860, G Loss: 0.6246
4/4 [==============================] - 0s 4ms/step
1
1/2 [==============>...............] - ETA: 0s

c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


2/2 [==============================] - 0s 4ms/step
Epoch 0: D Loss Real: 0.7083, D Loss Fake: 0.7668, G Loss: 0.6342
4/4 [==============================] - 0s 3ms/step
2


...

Epoch 0: D Loss Real: 0.6832, D Loss Fake: 0.7996, G Loss: 0.6247
4/4 [==============================] - 0s 2ms/step
55
2/2 [==============================] - 0s 3ms/step


c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 0: D Loss Real: 0.6778, D Loss Fake: 0.8030, G Loss: 0.6092
4/4 [==============================] - 0s 3ms/step
56
2/2 [==============================] - 0s 3ms/step


c:\Users\lalit\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 0: D Loss Real: 0.6946, D Loss Fake: 0.7878, G Loss: 0.6203
4/4 [==============================] - 0s 2ms/step


In [10]:
synthetic_df

,CROPS,TYPE_OF_CROP,SOIL,SOIL_PH,CROP_DURATION,WATERSOURCE,TEMP,WATER_REQUIRED,RELATIVE_HUMIDITY
0,rice,cereals,Alluvia or loamy and clayey soil,0.7,150.0,"irrigated,rainfall",0.08,2356.0,0.98
1,rice,cereals,Alluvia or loamy and clayey soil,0.6,150.0,"irrigated,rainfall",0.02,2370.0,0.99
2,rice,cereals,Alluvia or loamy and clayey soil,0.6,150.0,"irrigated,rainfall",0.04,2359.0,0.98
3,rice,cereals,Alluvia or loamy and clayey soil,0.7,150.0,"irrigated,rainfall",0.01,2383.0,0.99
4,rice,cereals,Alluvia or loamy and clayey soil,0.9,150.0,"irrigated,rainfall",0.03,2380.0,0.98
...,...,...,...,...,...,...,...,...,...
5695,small onion,bulbvegetables,Sandy loam,0.0,89.0,"irrigated,rainfall",0.84,698.0,0.99
5696,small onion,bulbvegetables,Sandy loam,0.0,89.0,"irrigated,rainfall",0.84,683.0,0.99
5697,small onion,bulbvegetables,Sandy loam,0.0,89.0,"irrigated,rainfall",0.80,678.0,0.99
5698,small onion,bulbvegetables,Sandy loam,0.0,89.0,"irrigated,rainfall",0.81,695.0,0.98


In [8]:
# Save synthetic data to CSV
synthetic_df.to_csv("synthetic_crop_data.csv")

In [ ]:

synthetic_df.to_csv("crop_data1.csv")